In [3]:
import math
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import matplotlib.lines as lines

#---------------------------------------------------conversions----------------------------------------------------#
#julian days from regular days
def juliandate(year, month, day, hour, minute, second):
        if year < 1801 or year > 2099:
            raise ValueError('Datetime must be between year 1801 and 2099')
        julian_datetime = 367 * year - int(7 * (year
                + int((month + 9) / 12.0)) / 4.0) + int(275
                * month / 9.0) + day + 1721013.5 + (hour
                + minute / 60.0 + second / 60**2) \
            / 24.0 - 0.5 * math.copysign(1, 100 * year + month
                - 190002.5) + 0.5
        return julian_datetime

#conversion of kilometers to AU 
def kmToAU(d):
    return d * 6.6845871226706E-9

#conversion of all dimensions of a vector from kilometers into AU 
def vKmToAU(v):
        return [kmToAU(d) for d in v]

#conversion of 
def kmpsToAUperGday(s):
        return s * 6.6845871226706E-9 / 1.9909836749106170445E-7
    
def vKmpsToAUperGday(v):
        return [kmpsToAUperGday(d) for d in v]

#conversion of degree form of Dec into decimalized form of Dec
def DMStoDeg (degrees, arcminutes, arcseconds):
    answer = 0 
    if math.copysign(1,degrees) < 0:
        answer = -degrees + arcminutes/60 + arcseconds/3600 
        return -answer
    else: 
        answer = degrees + arcminutes/60 + arcseconds/3600 
        return answer 

#conversion of a decimalized form of RA to hour form 
def RAdecimalToHMS (RA):
    hours = RA//15
    minutes = math.trunc((RA/15-hours)*60) 
    seconds = ((RA/15-hours)*60 - minutes)*60
    return (hours, minutes, seconds)

#conversion of degrees to radians 
def degtorad(deg):
    return deg*(2*math.pi)/180

#conversion of RA in hours into RA in degrees 
def HMStoDeg(hours, minutes, seconds):
        deg = 15 * (hours + (minutes + seconds / 60) / 60) % 360
        return deg

#conversion of decimalized form of Dec into the degree form of Dec
def DECdecimalToDMS (DEC):
    if math.copysign(1,DEC) < 0: 
        degrees = math.trunc(-DEC)
        arcminutes = math.trunc((-DEC-degrees)*60)
        arcseconds = ((-DEC-degrees)*60-arcminutes)*60
        return (-degrees, arcminutes, arcseconds)
    else: 
        degrees = math.trunc(DEC)
        arcminutes = math.trunc((DEC-degrees)*60)
        arcseconds = ((DEC-degrees)*60-arcminutes)*60
        return (degrees, arcminutes, arcseconds)

#-----------------------------------------------basic computations------------------------------------------------#
def anglefinder (sine, cosine):
    if sine > 0 and cosine > 0: 
        return np.arcsin(sine)
    if sine < 0 and cosine > 0:
        return 2*math.pi - math.acos(cosine)
    if sine < 0 and cosine < 0:
        return math.pi + np.arcsin(-sine)
    if sine > 0 and cosine < 0:
        return math.pi - np.arcsin(sine)
    if sine == 0 and cosine == 1:
        return 0
    if sine == 0 and cosine == -1:
        return math.pi
    if sine == 1 and cosine == 0:
        return math.pi/2
    if sine == -1 and cosine ==0:
        return 3*math.pi/2
    
def dot(v1, v2):
    return v1[0] * v2[0] + v1[1] * v2[1] + v1[2] * v2[2]

def cross(v1, v2):
    return [v1[1]*v2[2]-v1[2]*v2[1],v1[2]*v2[0]-v1[0]*v2[2],v1[0]*v2[1]-v1[1]*v2[0]]
    
def triple(a, b, c):
    return dot(a,cross(b,c))

def mag (n):
    ans = math.sqrt(n[0]**2+n[1]**2+n[2]**2)
    return ans

#methods of calculation
def mewton(f, df, i, e):
    guess = i
    while (True):
        new = guess - f(guess) / df(guess)
        if (abs(new - guess) < e):
            return newGuess
        guess = new
    
#------------------------------------------orbital determination elements------------------------------------------#
#calculation of angular momentum with r and v
def fh(r,v):
    return cross(r, v)

#calculation of the semimajor axis (a) with r and v
def fa(r,v):
    nr = mag(r)
    nv = mag(v)
    return (1/((2/nr)-(nv**2)))

#calculation of the eccentricity (a) with r and v
def fe(r,v):
    h = fh(r,v)
    nh = mag(h)
    return math.sqrt(1-nh**2/fa(r,v))
    
#calculation of inclination (i) with r and v
def inclination(r,v):
    h = fh(r,v)
    nh = mag(h)
    return math.acos(h[2]/nh) 

#calculation of the longitude of ascending node with r and v
def fomega(r,v):
    h = fh(r,v)
    nh = mag(h)
    inc = inclination(r,v)
    sinO = h[0]/(nh*math.sin(inc))
    cosO = -h[1]/(nh*math.sin(inc))
    return anglefinder(sinO, cosO)

#calculation of the angle between the line of ascending nodes and the position vector r (U) using r and v
def fU(r,v):
    inc = inclination(r,v)
    nr = mag(r)
    ome = fomega(r,v)
    sinU = r[2]/(nr*math.sin(inc))
    cosU = (r[0]*math.cos(ome)+r[1]*math.sin(ome))/nr
    return anglefinder(sinU, cosU)

#calculation of the true anomaly (nu) from r and v
def fnu(r,v):
    h = fh(r,v)
    cosgamma = (1/fe(r,v))*(((fa(r,v)*(1-fe(r,v)**2))/mag(r))-1)
    singamma = ((fa(r,v)*(1-fe(r,v)**2))/(mag(h)*fe(r,v)))*np.dot(r,v)/mag(r)
    return anglefinder(singamma,cosgamma)

#calculation of the argument of perihelion (smallomega) from using r and v
def smallomega(r,v):
    return (2*math.pi+fU(r,v)-fnu(r,v))%(2*math.pi)

#calculation of the eccentric anamoly (E) from using r and v
def fE(r,v):
    gangle = fnu(r,v)
    cosE = (fe(r,v)+math.cos(gangle))/(1+fe(r,v)*math.cos(gangle))
    sinE = (mag(r)*math.sin(gangle)/(fa(r,v)*math.sqrt(1-fe(r,v)**2)))
    return anglefinder(sinE,cosE)

#calculation of the mean anamoly (M) from using r and v
def fM(r,v):
    return fE(r,v) - fe(r,v)*math.sin(fE(r,v))

#calculation of the time of perihelion passage (T) from using r and v
def fT(t, r,v):
    return t - fM(r,v)*fa(r,v)**(3/2)/0.01720209895

#presentation of orbital elements in a dictionary format
def orbitalElements(r, v, t):
        h = fh(r, v)
        a = fa(r, v)
        e = fe(r, v)
        i = inclination(r,v)
        Omega = fomega(r, v)
        U = fU(r, v)
        nu = fnu(r, v)
        omega = smallomega(r, v)
        E = fE(r, v)
        M = fM(r, v)
        T = fT(t, r, v)
        return {"a": a,"e": e,"i": i,"Omega": Omega,"omega": omega,"T": T}

#------------------------------------Other Functions for of the Method of Gauss------------------------------------#
#Scalar Equation of Lagrange to Calculate the first guesses for the r2 values
def SEL(tau,sun2,rhohat2,Ds):
    A1 = tau[1]/tau[2]
    A3 = -tau[0]/tau[2]
    B1 = (A1/6)*(tau[2]**2-tau[1]**2)
    B3 = (A3/6)*(tau[2]**2-tau[0]**2)
    A = (A1*Ds[1]-Ds[2]+A3*Ds[3])/(-Ds[0])
    B = (B1*Ds[1]+B3*Ds[3])/(-Ds[0])
    E = -2*np.dot(rhohat2,sun2)
    F = mag(sun2)**2
    a = -(A**2+A*E+F)
    b = -(2*A*B+B*E)
    c = -(B**2)
    realans = []
    ans = np.polynomial.polynomial.polyroots([c,0,0,b,0,0,a,0,1])
    for r in ans:
            if (np.imag(r) == 0 and np.real(r) >= 0):
                realans.append(np.real(r))
    r2rho2s = [(r, A + B / r**3) for r in realans]
    return r2rho2s

#calculation of RA and Dec from r, v, t, and the sun to earth vector 
def getRaDec(r, v, t, earthSun):
        #value setting 
        a = a(r,v)
        e = e(r,v)
        T = (t,r,v)
        i = inclination(r,v)
        Omega = fomega(r,v)
        omega = smallomega(r,v)
        k = 0.01720209894
        u = 1
        n = k * sqrt(u / a**3)
        M = n * (t - T)
        
        E = mewton(lambda E: E - e * sin(E) - M, lambda E: 1 - e * math.cos(E), M, 10**(-10))
        orbitCoor = np.array([a * cos(E) - a * e, a * sqrt(1 - e**2) * sin(E), 0])
        ecliptic = np.array([
            [cos(Omega), -sin(Omega), 0],
            [sin(Omega), cos(Omega),  0],
            [0,          0,           1]
        ]).dot(np.array([
            [1, 0,      0      ],
            [0, cos(i), -sin(i)],
            [0, sin(i), cos(i) ]
        ]).dot(np.array([
            [cos(omega), -sin(omega), 0],
            [sin(omega), cos(omega),  0],
            [0,          0,           1]
        ]).dot(orbitCoor)))
        epsilon = c['eps']
        equatorial = c['eclToEq'].dot(ecliptic)

        rho = equatorial + earthSun
        rhohat = rho / Utils.mag(rho)
        DEC = asin(rhohat[2])
        RA = Utils.ang(rhohat[1] / cos(DEC), rhohat[0] / cos(DEC))
        return RA, DEC,mag(rho)

#conversion between rho and RA and Dec values 
def rho(RA, Dec):
    rho = [math.cos(RA)*math.cos(Dec),math.sin(RA)*math.cos(Dec),math.sin(Dec)] 
    return rho
    
#calculation of D values from RA, Dec, and sun vectors 
def DValues(rho1,rho2,rho3,Sun1,Sun2,Sun3):
    Do = dot(rho1,cross(rho2,rho3))
    D11 = dot(cross(Sun1,rho2),rho3)
    D12 = dot(cross(Sun2,rho2),rho3)
    D13 = dot(cross(Sun3,rho2),rho3)
    D21 = dot(cross(rho1,Sun1),rho3)
    D22 = dot(cross(rho1,Sun2),rho3)
    D23 = dot(cross(rho1,Sun3),rho3)
    D31 = dot(rho1,cross(rho2,Sun1))
    D32 = dot(rho1,cross(rho2,Sun2))
    D33 = dot(rho1,cross(rho2,Sun3))
    
    return [Do,D11,D12,D13,D21,D22,D23,D31,D32,D33]

#initial guesses estimates of f and g values
def fgStart(tau1, tau3, r2):
    u = 1/(r2**3)
    f1 = 1 - 0.5*u*tau1**2 
    g1 = tau1 - (1/6)*u*tau1**3 
    f3 = 1 - 0.5*u*tau3**2  
    g3 = tau3 - (1/6)*u*tau3**3 
    return(f1,f3,g1,g3)
    
#calculation of f and g 
def fg(tau1, tau3, r2, r2dot,flag):
    mr2 = mag(r2)
    u = 1/(mr2**3)
    z = dot(r2,r2dot)/mr2**2
    q = dot(r2dot,r2dot)/mr2**2
    if (flag == '3rd'):
        f1 = 1 - 0.5*u*tau1**2 + 0.5*u*z*tau1**3
        g1 = tau1 - (1/6)*u*tau1**3 
        f3 = 1 - 0.5*u*tau3**2 + 0.5*u*z*tau3**3 
        g3 = tau3 - (1/6)*u*tau3**3 
        return (f1,f3,g1,g3)
    elif (flag =='4th'):
        f1 = 1 - 0.5*u*tau1**2 + 0.5*u*z*tau1**3 + (1/24)*(3*u*q-15*u*z**2+u**2)*tau1**4
        g1 = tau1 - (1/6)*u*tau1**3 + 0.25*u*z*tau1**4
        f3 = 1 - 0.5*u*tau3**2 + 0.5*u*z*tau3**3 + (1/24)*(3*u*q-15*u*z**2+u**2)*tau3**4
        g3 = tau3 - (1/6)*u*tau3**3 + 0.25*u*z*tau3**4
        return (f1,f3,g1,g3)
    else:
        raise Exception("Invalid Flag")

#gets r2 from f and g 
def get_r_rdot_rhomags(rhohats, f1, f3, g1, g3, earthSuns):
    cs = [g3 / (f1 * g3 - g1 * f3), -1, -g1 / (f1 * g3 - g1 * f3)]
    d1 = -f3 / (f1 * g3 - g1 * f3)
    d3 = f1 / (f1 * g3 - g1 * f3)
    Ds = DValues(rhohats[0],rhohats[1],rhohats[2],earthSuns[0],earthSuns[1],earthSuns[2])
    rhos = [sum([cs[j] * Ds[1 + 3 * i + j] for j in range(3)]) / (cs[i] * Ds[0]) for i in range(3)]
    rhoVecs = [rhos[i] * np.array(rhohats[i]) for i in range(len(rhohats))]
    rs = [rhoVecs[i] - earthSuns[i] for i in range(len(rhoVecs))]
    r2dot = d1 * rs[0] + d3 * rs[2]
    return rs[1], r2dot, rhos

#corrects for time (subtracts distance over c)
def timeCorrection(t, rho):
    return t - rho / 173.14463267424032927762788003372296661973812438762204922212171566

#checks if values are alright 
def valid_r2rho2(r2rho2):
    r2, rho2 = r2rho2
    if (rho2 > 0.05):
        return True
    return False

#final orbital element combination
def getOrbitElems(ts, radecs, earthSuns):
        rhohats = [rho(ra, dec) for ra, dec in radecs]
        k = 0.01720209894
        taus = [k * (ts[0] - ts[1]), k * (ts[2] - ts[0]), k * (ts[2] - ts[1])]
        Ds = DValues(rhohats[0],rhohats[1],rhohats[2],earthSuns[0],earthSuns[1],earthSuns[2])
        r2rho2s = SEL(taus, earthSuns[1], rhohats[1], Ds)
        if (len(r2rho2s) == 0):
            raise Exception("no roots")
        for i in range(len(r2rho2s) - 1, -1, -1):
            r2rho2 = r2rho2s[i]
            if (not valid_r2rho2(r2rho2)):
                r2rho2s.pop(i)
        r2mag, rho2mag = r2rho2s[0] 
        f1, f3, g1, g3  = fgStart(taus[0], taus[2], r2mag)
        tCorr = []
        r2, r2dot = [0, 0, 0], [0, 0, 0]

        itr = 0
        while True:
            r2Prev, r2dotPrev = r2, r2dot
            r2, r2dot, rhomags = get_r_rdot_rhomags(rhohats, f1, f3, g1, g3, earthSuns)
            tCorr = [timeCorrection(ts[i], rhomags[i]) for i in range(len(ts))]
            errorR2 = mag([r2[i] - r2Prev[i] for i in range(len(r2))]) / mag(r2)
            errorR2dot = mag([r2dot[i] - r2dotPrev[i] for i in range(len(r2dot))]) /mag(r2dot)
            itr += 1
            if (errorR2 < 1E-10 and errorR2dot < 1E-10 or itr > 200):
                break
            taus = [k * (tCorr[0] - tCorr[1]), k * (tCorr[2] - tCorr[0]),k * (tCorr[2] - tCorr[1])]
            f1, f3, g1, g3 = fg(taus[0], taus[2], r2, r2dot, "4th")
        r2Ecl = np.array([[1, 0, 0], 
              [0, math.cos(23.4374 / 180 * math.pi), math.sin(23.4374 / 180 * math.pi)],
              [0, -math.sin(23.4374 / 180 * math.pi), math.cos(23.4374 / 180 * math.pi)]]).dot(r2)
        r2dotEcl = np.array([[1, 0, 0], 
              [0, math.cos(23.4374 / 180 * math.pi), math.sin(23.4374 / 180 * math.pi)],
              [0, -math.sin(23.4374 / 180 * math.pi), math.cos(23.4374 / 180 * math.pi)]]).dot(r2dot)
        orbElems = orbitalElements(r2Ecl, r2dotEcl, tCorr[1])
        print(r2Ecl, r2dotEcl)
        return orbElems

In [4]:
ts = []
tEph = 0
earthSunEph = []
radecs = []
earthSuns = [[-0.1109577975853843, 0.9272014379553272, 0.4018967671416078],
[-0.2930215712985966, 0.8932298297603838, 0.3871698996048206],
[-0.3412507591867596, 0.8786104024988677, 0.380830875421993]]

with open('Input.txt', 'r') as file:
    for line in file:
        lsplit = line.split(" ")
        yr = int(lsplit[0].strip())
        mth = int(lsplit[1].strip())
        day = int(lsplit[2].strip())
        hr, mnt, sec = [float(n) for n in lsplit[3].strip().split(":")]
        t = juliandate(yr, mth, day, hr, mnt, sec)

        raLst = [float(n) for n in lsplit[4].strip().split(":")]
        decLst = [float(n) for n in lsplit[5].strip().split(":")]
        ts.append(t)
        radecs.append((1 / (180/math.pi) *
                       HMStoDeg(raLst[0], raLst[1], raLst[2]),
                       1 / (180/math.pi) *
                       DMStoDeg(decLst[0], decLst[1], decLst[2])))
getOrbitElems(ts, radecs, earthSuns)

[ 0.4366421  -1.36618462  0.1674045 ] [ 0.70017072 -0.09749085 -0.38078946]


{'a': 1.3508114450416537,
 'e': 0.3300865769055278,
 'i': 0.5861093166569967,
 'Omega': 2.0568581691257988,
 'omega': 0.8310700689376773,
 'T': 2459636.6486577606}